**Imports**

In [5]:
import subprocess
import os

**1. Conversions**



In [6]:
def convert_to_vp8(input_path, output_path="output_vp8.webm"):
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libvpx",
        "-b:v", "1M",
        "-c:a", "libvorbis",
        output_path
    ]
    subprocess.run(cmd, check=True)
    print(f"VP8 conversion complete: {output_path}")

def convert_to_vp9(input_path, output_path="output_vp9.webm"):
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libvpx-vp9",
        "-b:v", "1M",
        "-c:a", "libvorbis",
        output_path
    ]
    subprocess.run(cmd, check=True)
    print(f"VP9 conversion complete: {output_path}")

def convert_to_h265(input_path, output_path="output_h265.mp4"):
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libx265",
        "-crf", "28",
        "-c:a", "aac",
        'b:a', '128k',
        output_path
    ]
    subprocess.run(cmd, check=True)
    print(f"H265 conversion complete: {output_path}")


def convert_to_av1(input_path, output_path="output_av1.mp4"):
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-c:v", "libaom-av1",
        "-crf", "30",
        'b:v', '0',
        '-strict', 'experimental',
        output_path
    ]
    subprocess.run(cmd, check=True)
    print(f"AV1 conversion complete: {output_path}")

def convert_to_all(input_path):
    convert_to_vp8(input_path)
    convert_to_vp9(input_path)
    convert_to_h265(input_path)
    convert_to_av1(input_path)
    

In [ ]:
convert_to_all("input.mov")

**Encoding Ladder**



In [15]:
def encode_ladder(input_path, codec="h265", resolutions=None, bitrates=None):
    if resolutions is None:
        resolutions = ['1920:1080', '1280:720', '854:480', '640:360']
    if bitrates is None:
        bitrates = ['1000k', '500k', '250k', '125k']
    
    ext = '.mp4' if codec in ['h265', 'av1'] else '.webm'

    for resolution, bitrate in zip(resolutions, bitrates):
        output_path = f"output_{codec}_{resolution.replace(':', 'x')}_{bitrate}.{ext.lstrip('.')}"
        cmd = [
            'ffmpeg',
            '-i', input_path,
            '-vf', f'scale={resolution}',
            '-b:v', bitrate,
            '-y'  # Overwrite output files without asking
        ]
        if codec == 'h265':
            cmd.extend([
                '-c:v', 'libx265',
                '-preset', 'fast',  # Faster encoding
                '-c:a', 'aac',
                '-b:a', '128k'
            ])
        elif codec == 'vp8':
            cmd.extend([
                '-c:v', 'libvpx',
                '-c:a', 'libvorbis'
            ])
        elif codec == 'vp9':
            cmd.extend([
                '-c:v', 'libvpx-vp9',
                '-c:a', 'libvorbis'
            ])
        elif codec == 'av1':
            cmd.extend([
                '-c:v', 'libaom-av1',
                '-strict', 'experimental'
            ])
        cmd.append(output_path)
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            print(f"Encoding complete: {output_path}")
        except subprocess.CalledProcessError as e:
            print(f"Error: {e.stderr}")
            raise

In [ ]:
encode_ladder("input.mov")